# Text generation with Pretrained GPT2 models from Hugging Face on Amazon SageMaker
## The Poetry of NLP

You’ve just been hired by the Chicago Tribune to start a new poetry column. Congrats! The catch? You need to write a new poem every day. And it can’t just be any old string of syllables, you need it to be fresh, authentic, to resonate with the times and carry a sense of rhyme. You need it to delight your readers, to drive up the Tribune’s daily readership numbers and grow their social media presence. How are you going to accomplish this? With the help of Hugging Face and NLP models on SageMaker of course! 

#### In this notebook, we'll execute the following steps.

1. Use the Hugging Face transformfers SDK to download pretrained NLP models and test them locally.
2. Select a dataset from among our favorite authors.
3. Finetune the pretrained model using SageMaker training.
4. Deploy the model into S3.
5. Trigger a pipeline to test and deploy the model onto a multi-container endpoint.
6. Test your multi-model endpoint locally to write poetry and text in the style of your favorite authors. 

Please note, this notebook was built on SageMaker Studio, using an ml.t3.medium kernel gatway application, and the Python 3.6 PyTorch 1.8 CPU Jupyter Kernel.

### Step 0. Install the transformers SDK locally.

In [ ]:
%%writefile requirements.txt 

transformers==4.6.1

In [ ]:
!pip install -r requirements.txt

### Step 1. Download a pretrained GPT2 model and test locally.
We're using the Transformers SDK syntax available on the model card here: https://huggingface.co/gpt2 

To make this model even better, we'll use a version of GPT2 that **has already been finetuned to generate poetry!**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

poem_gpt = "ismaelfaro/gpt2-poems.en"

tokenizer = AutoTokenizer.from_pretrained(poem_gpt)

base_model = AutoModelForCausalLM.from_pretrained(poem_gpt)

In [ ]:
from transformers import set_seed

def get_outputs(sample_outputs, tokenizer):
    # takes a tokenizer, and raw output from the model, decodes these and formats nicely
    rt = []

    print("Output:\n" + 100 * '-')
    for i, sample_output in enumerate(sample_outputs):
        txt = tokenizer.decode(sample_output, skip_special_tokens = True)
        print("{}: {}...".format(i, txt))
        print('')
        rt.append(txt)
        
    return rt

# setting the seed helps us ensure reproducibility. when the seed is consistent, we know the model results will be consistent
set_seed(42)

text = "A rose by any other name"

input_ids = tokenizer.encode(text, return_tensors = 'pt')

sample_outputs = base_model.generate(input_ids,
                                do_sample = True, 
                                max_length = 70,
                               num_return_sequences = 5)                              

generic_outputs = get_outputs(sample_outputs, tokenizer)

Interesting and entertaining! Clearly this model knows the form of poetry. It is obviously generating short lines, with a newline, and it seems to pick up some interesting concepts. Now, let's see if we can fine-tune this poem writer to fit the style of an author we have in mind.

### Step 2. Fine-tune the GPT2 Poem model with Anne Bradstreet.
Now, we're going to fine-tune this model using another, much smaller, dataset. Then later we'll use a text classifier trained to evaluate this style of writer, and see how well our new text performs!

If you're curious, take a look at some of the top authors in the English language available through this open domain site.
https://www.public-domain-poetry.com/topauthors.php 

For the purposes of this workshop we'll stick to the longer poem pasted below. On your time time, outside of the workshop, if you'd like to modify this to work with a different text you are welcome to do so.

Poke around at some of the available poems, and copy and paste what you like into this `train.txt` file below. We'll format that for finetuning GPT2 in the next step. In this notebook we're using a poem from Anne Bradstreet, a North American writer from the 17th Century.

You may not have known this, but Anne Bradstreet was the first writer to be published in the North America!

In [ ]:
%%writefile train.txt

A Dialogue Between Old England And New

    By Anne Bradstreet

    New England.

    Alas, dear Mother, fairest Queen and best,
    With honour, wealth, and peace happy and blest,
    What ails thee hang thy head, and cross thine arms,
    And sit i� the dust to sigh these sad alarms?
    What deluge of new woes thus over-whelm
    The glories of thy ever famous Realm?
    What means this wailing tone, this mournful guise?
    Ah, tell thy Daughter; she may sympathize.

    Old England.

    Art ignorant indeed of these my woes,
    Or must my forced tongue these griefs disclose,
    And must my self dissect my tatter�d state,
    Which Amazed Christendom stands wondering at?
    And thou a child, a Limb, and dost not feel
    My weak�ned fainting body now to reel?
    This physic-purging-potion I have taken
    Will bring Consumption or an Ague quaking,
    Unless some Cordial thou fetch from high,
    Which present help may ease my malady.
    If I decease, dost think thou shalt survive?
    Or by my wasting state dost think to thrive?
    Then weigh our case, if �t be not justly sad.
    Let me lament alone, while thou art glad.

    New England.

    And thus, alas, your state you much deplore
    In general terms, but will not say wherefore.
    What Medicine shall I seek to cure this woe,
    If th� wound�s so dangerous, I may not know?
    But you, perhaps, would have me guess it out.
    What, hath some Hengist like that Saxon stout
    By fraud and force usurp�d thy flow�ring crown,
    Or by tempestuous Wars thy fields trod down?
    Or hath Canutus, that brave valiant Dane,
    The regal peaceful Sceptre from thee ta�en?
    Or is �t a Norman whose victorious hand
    With English blood bedews thy conquered Land?
    Or is �t intestine Wars that thus offend?
    Do Maud and Stephen for the Crown contend?
    Do Barons rise and side against their King,
    And call in Foreign aid to help the thing?
    Must Edward be depos�d? Or is �t the hour
    That second Richard must be clapp�d i� th� Tower?
    Or is it the fatal jar, again begun,
    That from the red, white pricking Roses sprung?
    Must Richmond�s aid the Nobles now implore
    To come and break the tushes of the Boar?
    If none of these, dear Mother, what�s your woe?
    Pray, do not fear Spain�s bragging Armado.
    Doth your Ally, fair France, conspire your wrack,
    Or doth the Scots play false behind your back?
    Doth Holland quit you ill for all your love?
    Whence is this storm, from Earth or Heaven above?
    Is �t drought, is �t Famine, or is �t Pestilence?
    Dost feel the smart, or fear the consequence?
    Your humble Child entreats you shew your grief.
    Though Arms nor Purse she hath for your relief�
    Such is her poverty,�yet shall be found
    A suppliant for your help, as she is bound.

    Old England.

    I must confess some of those Sores you name
    My beauteous Body at this present maim,
    But foreign Foe nor feigned friend I fear,
    For they have work enough, thou knowest, elsewhere.
    Nor is it Alcie�s son and Henry�s Daughter
    Whose proud contention cause this slaughter;
    Nor Nobles siding to make John no King,
    French Louis unjustly to the Crown to bring;
    No Edward, Richard, to lose rule and life,
    Nor no Lancastrians to renew old strife;
    No Crook-backt Tyrant now usurps the Seat,
    Whose tearing tusks did wound, and kill, and threat.
    No Duke of York nor Earl of March to soil
    Their hands in Kindred�s blood whom they did foil;
    No need of Tudor Roses to unite:
    None knows which is the Red or which the White.
    Spain�s braving Fleet a second time is sunk.
    France knows how of my fury she hath drunk
    By Edward third and Henry fifth of fame;
    Her Lilies in my Arms avouch the same.
    My Sister Scotland hurts me now no more,
    Though she hath been injurious heretofore.
    What Holland is, I am in some suspense,
    But trust not much unto his Excellence.
    For wants, sure some I feel, but more I fear;
    And for the Pestilence, who knows how near?
    Famine and Plague, two sisters of the Sword,
    Destruction to a Land doth soon afford.
    They�re for my punishments ordain�d on high,
    Unless thy tears prevent it speedily.
    But yet I answer not what you demand
    To shew the grievance of my troubled Land.
    Before I tell the effect I�ll shew the cause,
    Which are my sins�the breach of sacred Laws:
    Idolatry, supplanter of a N ation,
    With foolish superstitious adoration,
    Are lik�d and countenanc�d by men of might,
    The Gospel is trod down and hath no right.
    Church Offices are sold and bought for gain
    That Pope had hope to find Rome here again.
    For Oaths and Blasphemies did ever ear
    From Beelzebub himself such language hear?
    What scorning of the Saints of the most high!
    What injuries did daily on them lie!
    What false reports, what nick-names did they take,
    Not for their own, but for their Master�s sake!
    And thou, poor soul, wast jeer�d among the rest;
    Thy flying for the Truth I made a jest.
    For Sabbath-breaking and for Drunkenness
    Did ever Land profaneness more express?
    From crying bloods yet cleansed am not I,
    Martyrs and others dying causelessly.
    How many Princely heads on blocks laid down
    For nought but title to a fading Crown!
    �Mongst all the cruelties which I have done,
    Oh, Edward�s Babes, and Clarence�s hapless Son,
    O Jane, why didst thou die in flow�ring prime?�
    Because of Royal Stem, that was thy crime.
    For Bribery, Adultery, for Thefts, and Lies
    Where is the Nation I can�t paralyze?
    With Usury, Extortion, and Oppression,
    These be the Hydras of my stout transgression;
    These be the bitter fountains, heads, and roots
    Whence flow�d the source, the sprigs, the boughs, and fruits.
    Of more than thou canst hear or I relate,
    That with high hand I still did perpetrate,
    For these were threat�ned the woeful day
    I mocked the Preachers, put it fair away.
    The Sermons yet upon record do stand
    That cried destruction to my wicked Land.
    These Prophets� mouths (all the while) was stopt,
    Unworthily, some backs whipt, and ears crept;
    Their reverent cheeks bear the glorious marks
    Of stinking, stigmatizing Romish Clerks;
    Some lost their livings, some in prison pent,
    Some grossly fined, from friends to exile went:
    Their silent tongues to heaven did vengeance cry,
    Who heard their cause, and wrongs judg�d righteously,
    And will repay it sevenfold in my lap.
    This is fore-runner of my after-clap.
    Nor took I warning by my neighbors� falls.
    I saw sad Germany�s dismantled walls,
    I saw her people famish�d, Nobles slain,
    Her fruitful land a barren heath remain.
    I saw (unmov�d) her Armies foil�d and fled,
    Wives forc�d, babes toss�d, her houses calcined.
    I saw strong Rochelle yield�d to her foe,
    Thousands of starved Christians there also.
    I saw poor Ireland bleeding out her last,
    Such cruelty as all reports have past.
    Mine heart obdurate stood not yet aghast.
    Now sip I of that cup, and just �t may be
    The bottom dregs reserved are for me.

    New England.

    To all you�ve said, sad mother, I assent.
    Your fearful sins great cause there �s to lament.
    My guilty hands (in part) hold up with you,
    A sharer in your punishment�s my due.
    But all you say amounts to this effect,
    Not what you feel, but what you do expect.
    Pray, in plain terms, what is your present grief?
    Then let�s join heads and hands for your relief.

    Old England.

    Well, to the matter, then. There�s grown of late
    �Twixt King and Peers a question of state:
    Which is the chief, the law, or else the King?
    One saith, it�s he; the other, no such thing.
    My better part in Court of Parliament
    To ease my groaning land shew their intent
    To crush the proud, and right to each man deal,
    To help the Church, and stay the Common-Weal.
    So many obstacles comes in their way
    As puts me to a stand what I should say.
    Old customs, new Prerogatives stood on.
    Had they not held law fast, all had been gone,
    Which by their prudence stood them in such stead
    They took high Strafford lower by the head,
    And to their Laud be �t spoke they held �n th� Tower
    All England�s metropolitan that hour.
    This done, an Act they would have passed fain
    No prelate should his Bishopric retain.
    Here tugg�d they hard indeed, for all men saw
    This must be done by Gospel, not by law.
    Next the Militia they urged sore.
    This was denied, I need not say wherefore.
    The King, displeased, at York himself absents.
    They humbly beg return, shew their intents.
    The writing, printing, posting to and fro,
    Shews all was done; I�ll therefore let it go.
    But now I come to speak of my disaster.
    Contention�s grown �twixt Subjects and their Master,
    They worded it so long they fell to blows,
    That thousands lay on heaps. Here bleeds my woes.
    I that no wars so many years have known
    Am now destroy�d and slaughter�d by mine own.
    But could the field alone this strife decide,
    One battle, two, or three I might abide,
    But these may be beginnings of more woe�
    Who knows, the worst, the best may overthrow!
    Religion, Gospel, here lies at the stake,
    Pray now, dear child, for sacred Zion�s sake,
    Oh, pity me in this sad perturbation,
    My plundered Towns, my houses� devastation,
    My ravisht virgins, and my young men slain,
    My wealthy trading fallen, my dearth of grain.
    The seedtime�s come, but Ploughman hath no hope
    Because he knows not who shall inn his crop.
    The poor they want their pay, their children bread,
    Their woful mothers� tears unpitied.
    If any pity in thy heart remain,
    Or any child-like love thou dost retain,
    For my relief now use thy utmost skill,
    And recompense me good for all my ill.

    New England.

    Dear mother, cease complaints, and wipe your eyes,
    Shake off your dust, cheer up, and now arise.
    You are my mother, nurse, I once your flesh,
    Your sunken bowels gladly would refresh.
    Your griefs I pity much but should do wrong,
    To weep for that we both have pray�d for long,
    To see these latter days of hop�d-for good,
    That Right may have its right, though �t be with blood.
    After dark Popery the day did clear;
    But now the Sun in�s brightness shall appear.
    Blest be the Nobles of thy Noble Land
    With (ventur�d lives) for truth�s defence that stand.
    Blest be thy Commons, who for Common good
    And thy infringed Laws have boldly stood.
    Blest be thy Counties, who do aid thee still
    With hearts and states to testify their will.
    Blest be thy Preachers, who do cheer thee on.
    Oh, cry: the sword of God and Gideon!
    And shall I not on them wish Mero�s curse
    That help thee not with prayers, arms, and purse?
    And for my self, let miseries abound
    If mindless of thy state I e�er be found.
    These are the days the Church�s foes to crush,
    To root out Prelates, head, tail, branch, and rush.
    Let�s bring Baal�s vestments out, to make a fire,
    Their Mitres, Surplices, and all their tire,
    Copes, Rochets, Croziers, and such trash,
    And let their names consume, but let the flash
    Light Christendom, and all the world to see
    We hate Rome�s Whore, with all her trumpery.
    Go on, brave Essex, shew whose son thou art,
    Not false to King, nor Country in thy heart,
    But those that hurt his people and his Crown,
    By force expel, destroy, and tread them down.
    Let Gaols be fill�d with th� remnant of that pack,
    And sturdy Tyburn loaded till it crack.
    And ye brave Nobles, chase away all fear,
    And to this blessed Cause closely adhere.
    O mother, can you weep and have such Peers?
    When they are gone, then drown your self in tears,
    If now you weep so much, that then no more
    The briny Ocean will o�erflow your shore.
    These, these are they (I trust) with Charles our king,
    Out of all mists such glorious days will bring
    That dazzled eyes, beholding, much shall wonder
    At that thy settled Peace, thy wealth, and splendour,
    Thy Church and Weal establish�d in such manner
    That all shall joy that thou display�dst thy banner,
    And discipline erected so, I trust,
    That nursing Kings shall come and lick thy dust.
    Then Justice shall in all thy Courts take place
    Without respect of persons or of case.
    Then bribes shall cease, and suits shall not stick long,
    Patience and purse of Clients for to wrong.
    Then High Commissions shall fall to decay,
    And Pursuivants and Catchpoles want their pay.
    So shall thy happy Nation ever flourish,
    When truth and righteousness they thus shall nourish.
    When thus in Peace, thine Armies brave send out
    To sack proud Rome, and all her vassals rout.
    There let thy name, thy fame, and valour shine,
    As did thine Ancestors� in Palestine,
    And let her spoils full pay with int�rest be
    Of what unjustly once she poll�d from thee.
    Of all the woes thou canst let her be sped,
    Execute to th� full the vengeance threatened.
    Bring forth the beast that rul�d the world with�s beck,
    And tear his flesh, and set your feet on�s neck,
    And make his filthy den so desolate
    To th� �stonishment of all that knew his state.
    This done, with brandish�d swords to Turkey go,�
    (For then what is it but English blades dare do?)
    And lay her waste, for so�s the sacred doom,
    And do to Gog as thou hast done to Rome.
    Oh Abraham�s seed, lift up your heads on high,
    For sure the day of your redemption�s nigh.
    The scales shall fall from your long blinded eyes,
    And him you shall adore who now despise.
    Then fullness of the Nations in shall flow,
    And Jew and Gentile to one worship go.
    Then follows days of happiness and rest.
    Whose lot doth fall to live therein is blest.
    No Canaanite shall then be found �n th� land,
    And holiness on horses� bells shall stand.
    If this make way thereto, then sigh no more,
    But if at all thou didst not see �t before.
    Farewell, dear mother; Parliament, prevail,
    And in a while you�ll tell another tale.


### Step 3. Format your training data for Hugging Face on Amazon SageMaker.
Now, let's parse your training data to format it for finetuning GPT2 and training on Hugging Face. 

In [ ]:
data = []

with open('train.txt') as f:
    for row in f.readlines():
        d = row.strip()
        if len(d) > 1:
            data.append(d)

In [ ]:
print ('Found {} valid objects in the training data.'.format(len(data)))

In [ ]:
print (data[:10])

In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket() 

train_file_name = 'train.txt'
s3_train_data = 's3://{}/gpt2/{}'.format(bucket, train_file_name)

!aws s3 cp {train_file_name} {s3_train_data}

In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFace, TrainingCompilerConfig

# gets role for executing training job
role = sagemaker.get_execution_role()
hyperparameters = {
    'model_name_or_path':"ismaelfaro/gpt2-poems.en",
    'output_dir':'/opt/ml/model',
    'do_train':True,
    'train_file': '/opt/ml/input/data/train/{}'.format(train_file_name),
    'num_train_epochs': 5,
    # set batch size to 22 if using SM training compiler
    "per_device_train_batch_size": 64,
    # add your remaining hyperparameters
    # more info here https://github.com/huggingface/transformers/tree/v4.6.1/examples/pytorch/language-modeling
}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
    entry_point='run_clm.py',
    source_dir='./examples/pytorch/language-modeling',
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    git_config=git_config,
    transformers_version='4.11.0',
    pytorch_version='1.9.0',
    py_version='py38',
    hyperparameters = hyperparameters,
    # pass the training compiler config to speed up your job
    compiler_config=TrainingCompilerConfig(),
    environment = {'GPU_NUM_DEVICES':'1'},
    disable_profiler = True,
    debugger_hook_config = False
)

# starting the train job
# should take about 13 minutes to run on current settings
huggingface_estimator.fit({'train':s3_train_data}, wait = True)

### Step 4. Test your trained model locally

In [ ]:
from sagemaker.huggingface import HuggingFace
import time

# redefining if you need to restart your kernel 
# huggingface_estimator = HuggingFace.attach('huggingface-pytorch-trcomp-training-2022-03-13-18-29-53-098')
try:
    s3_model_data = huggingface_estimator.model_data
    local_model_path = 'gpt2_finetuned'
    
except:
    time.sleep(5)
    s3_model_data = huggingface_estimator.model_data
    local_model_path = 'gpt2_finetuned'
    

In [ ]:
!mkdir {local_model_path}
!aws s3 cp {s3_model_data} {local_model_path}
!tar -xvf {local_model_path}/model.tar.gz -C {local_model_path}
!rm {local_model_path}/model.tar.gz

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# optional - rerun this if you need to restart your kernel. We are actually using the same tokenizer from before
tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained(local_model_path)

In [ ]:
# step to make sure we can run inference with this model locally
model.eval()

In [ ]:
from transformers import set_seed

set_seed(42)

text = "A rose by any other name "
input_ids = tokenizer.encode(text, return_tensors = 'pt')

sample_outputs = model.generate(input_ids,
                                do_sample = True, 
                                max_length = 70,
                               num_return_sequences = 5)                              
    
bradsteet_raw = get_outputs(sample_outputs, tokenizer)

Interesting, it certainly looks different. Let's see if we can modify this output using different paramters to invoke the trained model.

In [ ]:
sample_outputs = model.generate(input_ids, 
                             max_length=70,
                             do_sample=True, 
                             # only pick tokens at and above this probability level
                             top_p=0.85,
                             # only pick from this many tokens
                             top_k=200,
                             num_return_sequences = 5)                              


bradstreet_top_85 = get_outputs(sample_outputs, tokenizer)


Wow! Quite a difference - not all of these seem very much like Bradstreet, and just much more generic. Yet the logical coherence on some of them is strong. Let's try it again with a smaller top_k and top_p.

In [ ]:
sample_outputs = model.generate(input_ids, 
                             max_length=70,
                             do_sample=True, 
                             # only pick tokens at and above this probability level
                             top_p=0.95,
                             # only pick from this many tokens
                             top_k=110,
                             num_return_sequences = 5)                              


bradstreet_top_95 = get_outputs(sample_outputs, tokenizer)


Interesting - under these terms the model seems even more generic. You can still pick up a hint of that very old English style of writing, and yet the social media base terms come even more to the surface.

### Step 5. Load a Text Classifier to Quantify Our Generated Text
Now, we're going to use another model from the HF Hub. This time it's a text classifier, built specifically to give a strong signal for whether or not our text seems like it's in the style of Anne Bradstreet.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

anne_model_name = 'edubz/anne_bradstreet'

anne_tokenizer = AutoTokenizer.from_pretrained(anne_model_name)

anne_clf = AutoModelForSequenceClassification.from_pretrained(anne_model_name)

In [ ]:
from scipy.special import softmax

def invoke_locally(text, anne_clf, anne_tokenizer):
    
    input_ids = anne_tokenizer(text, return_tensors = 'pt')

    output = anne_clf(**input_ids)

    logits = output['logits'].detach().numpy().tolist()[0]

    res = softmax(logits).tolist()

    conf = max(res)

    label = res.index(conf)
    
    if label == 0:
        label_str = 'Not Anne'
    elif label == 1:
        label_str = 'Anne'
    
    return {'confidence': conf, 'label':label_str }

In [ ]:
invoke_locally("Alas, dear Mother, fairest Queen and best", anne_clf, anne_tokenizer)

In [ ]:
invoke_locally("A rose by any other name", anne_clf, anne_tokenizer)

In [ ]:
invoke_locally("< paste your generated text here >", anne_clf, anne_tokenizer)

Now, run some tests of your own. Try different invocation parameters. What seems to get you the highest Anne scores?

### Step 4. Deploy your fine-tuned model onto a SageMaker multi-model endpoint
Now, let's deploy this model onto SageMaker. In particular we will trigger a pipeline to update an existing multi-model endpoint, and then invoke our model from that endpoint. 

We'll also list all available models from that endpoint, and test out generating text with each of these. Who knows, maybe we'll stumble on something good enough for the Tribune!

In [ ]:
bradstreet_path = 'gpt2-bradstreet-model'

In [ ]:
model.save_pretrained('{}/'.format(bradstreet_path))
tokenizer.save_pretrained('{}/'.format(bradstreet_path))

In [ ]:
import tarfile 

b_model_name = '{}.tar.gz'.format(bradstreet_path)

with tarfile.open(b_model_name, 'w:gz') as f:
    f.add('gpt2-model/',arcname='.')
f.close()

prefix = 'gpt2-hf-workshop/gpt2-test'

In [ ]:
! tar -ztvf {b_model_name}

In [ ]:
!aws s3 cp {b_model_name} s3://{bucket}/{prefix}/{b_model_name}

### Step 5. Test your fine-tuned model on SageMaker multi-model endpoint

In [ ]:
import boto3

client = boto3.client('sagemaker')

endpoints = client.list_endpoints()

In [ ]:
for e in endpoints['Endpoints']:
    name = e['EndpointName']
    if 'mme' in name:
        print (name)
        mme_name = name

In [ ]:
predictor = sagemaker.predictor.Predictor(endpoint_name = mme_name, sagemaker_session=sess)
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [ ]:
# the first time you invoke a model on an MME it will take longer to respond b/c the model is being copied from S3
predictor.predict({"inputs":'A rose by any other name'}, target_model=b_model_name)

In [ ]:
# predictor = point to MME predictor here 
predictor.predict({"inputs":'A rose by any other name'}, target_model=b_model_name)

### Step 7. Write poetry for the Chicago Tribune
Now - select your favorite lines from each output from GPT, and pass it in to the model. Feel free to modify the parameters using kwargs. When you are finished, you can submit your poem to our GitHub workshop page!

**Please note** every time you invoke a new model via MME AWS is copying the model artifact from S3 to the SageMaker endpoint. That means **expect a big time delay whenever you invoke a new model.** 

One way to get around that is with model compilation, ie running SageMaker Neo to decrease the size, and thereby the runtime, of that model.

In the poem below, I manually copied my favorite line from each output of the model, and fed it in to the generator. I manually pasted all of my favorites into the markdown file you see below.

---

### My poem - A rose by any other model

A rose by any other name has many meanings. <br />
When all that has been presented to us is a form of exaggeration. <br />
The language will not preserve. <br />
However, the old idea of he who has no business vainly passing by without any other <br />
Some unending mizzen, deceived and deceived, seems ever more absurd and likely to harm our time. <br />
We tuck his back into the sea which is on the plain almost as soon as we lose sight of him. <br />
A mariner shall pass. <br />
And I may leave nothing to thee till thou return, for as I said, My hand am strong when thou shouldst require it. <br />
This comes out of Kant\'s conviction that we have nothing in our minds<br />

In [ ]:
text = 'A rose by any other name has many meanings.'
predictor.predict({"inputs":text, 
                   'parameters':{'max_length':70,
                             'do_sample':True, 
                             # only pick tokens at and above this probability level
                             'top_p':0.99,
                             # only pick from this many tokens
                             'top_k':600}}, 
                  target_model=b_model_name)

In [ ]:
text = 'However, the old idea of he who has no business vainly passing by without any other means'

predictor.predict({"inputs":text, 
                   'parameters':{'max_length':70,
                             'do_sample':True, 
                             # only pick tokens at and above this probability level
                             'top_p':0.99,
                             # only pick from this many tokens
                             'top_k':600}}, 
                  target_model=b_model_name)

In [ ]:
text = 'A mariner shall pass'

predictor.predict({"inputs":text, 
                   'parameters':{'max_length':70,
                             'do_sample':True, 
                             # only pick tokens at and above this probability level
                             'top_p':0.99,
                             # only pick from this many tokens
                             'top_k':600}}, 
                  target_model=b_model_name)

In [ ]:
text = 'A rose by any other model'

predictor.predict({"inputs":text, 
                   'parameters':{'max_length':70,
                             'do_sample':True, 
                             # only pick tokens at and above this probability level
                             'top_p':0.99,
                             # only pick from this many tokens
                             'top_k':100}}, 
                  target_model=b_model_name)

### Optional - use a pretrained GPTJ

Use the Endpoints notebook in this repository to deploy and test a GPT-J 6B endpoint. Compare the generation to that of your fine-tuned GPT-2 model. Add some of the lines to your poem if you like!

### Optional- Use Other Pretrained Models and Hugging Face Datasets

**Available pretrained models and datasets from Hugging Face**

**Datasets**
- Shakespeare:
    - https://huggingface.co/datasets/tiny_shakespeare 

**Pretrained models**
- Chinese Poetry:
    - https://huggingface.co/caixin1998/chinese-poetry-gpt2 
- Hebrew Poetry:
    - https://huggingface.co/Norod78/hebrew_poetry-gpt_neo-small 
- Arabic Poetry:
    - https://huggingface.co/akhooli/gpt2-small-arabic-poetry 
- Russian Poetry:
    - https://huggingface.co/TuhinColumbia/russianpoetrymany 
- Persian Poetry:
    - https://huggingface.co/mitra-mir/BERT-Persian-Poetry 
- Italian Poetry:
     - https://huggingface.co/TuhinColumbia/italianpoetrymany 

# Conclusion - Use Hugging Face and Text Generation on Amazon SageMaker for Your Organization
Now that you've learned how to test, finetune, deploy and utilize a text generation model on SageMaker, let's understand how to apply that within your organzation.

First, think to yourself, does my organization already produce a lot of written text? Do we write documentation, scripts, blog posts, documents, answers to questions, customer messaging, etc? Odds are, you do. 

Then, ask yourself, where do we already have a large volume of written text I can easily access? That may be your existing public documentation, your existing blog posts, etc. First, run through this notebook and use some of your own data to finetune a GPT model. See how well that performs, then consider scaling to large models, including GPT-J. 

If you really aren't seeing the performance you want, [consider training a model from scratch!](https://github.com/nlp-with-transformers/notebooks/blob/main/10_transformers-from-scratch.ipynb )

Look at this and other examples within [Hugging Face's SageMaker example notebooks](https://github.com/huggingface/notebooks/tree/master/sagemaker), and similar examples on the [SageMaker repository!](https://github.com/aws/amazon-sagemaker-examples/search?q=hugging+face)

Remember that in order to get the best performance we **combined a variety of computer-generated and human-discriminated approaches**. Further work could improve on this by training discriminator NLP models, such as text classifiers in certain styles, to make the generation and improvement process even faster.